Usando los dataframe de Dask, prepararemos un notebook para desarrollar los siguientes
puntos

> Cargaremos el conjunto de datos en un dataframe

In [21]:
import dask.dataframe as dd

In [22]:
df1 = dd.read_csv("data/air_traffic_data.csv")
df2 = dd.read_csv("data/ABT_CALIDAD_AIRE.csv")
df = dd.merge(df1, df2)
df.head()

,Activity Period,Operating Airline,Operating Airline IATA Code,Published Airline,Published Airline IATA Code,GEO Summary,GEO Region,Activity Type Code,Price Category Code,Terminal,...,chance_of_rain,will_it_snow,chance_of_snow,vis_km,festivo,anyo,mes,dia,hora,diasemana
0,200507,ATA Airlines,TZ,ATA Airlines,TZ,Domestic,US,Deplaned,Low Fare,Terminal 1,...,0,0,0,10.0,0,2017,10,1,0,1
1,200507,ATA Airlines,TZ,ATA Airlines,TZ,Domestic,US,Enplaned,Low Fare,Terminal 1,...,0,0,0,10.0,0,2017,10,1,1,1
2,200507,ATA Airlines,TZ,ATA Airlines,TZ,Domestic,US,Thru / Transit,Low Fare,Terminal 1,...,0,0,0,10.0,0,2017,10,1,2,1
3,200507,Air Canada,AC,Air Canada,AC,International,Canada,Deplaned,Other,Terminal 1,...,0,0,0,10.0,0,2017,10,1,3,1
4,200507,Air Canada,AC,Air Canada,AC,International,Canada,Enplaned,Other,Terminal 1,...,0,0,0,10.0,0,2017,10,1,4,1


He metido todos los datos en un mismo dataframe

> ¿Cuántas compañías diferentes aparecen en el fichero?

In [23]:
df["Operating Airline"].unique().compute()

0            ATA Airlines
1             Air Canada 
2               Air China
3              Air France
4         Air New Zealand
             ...         
72         Etihad Airways
73         China Southern
74       Turkish Airlines
75    COPA Airlines, Inc.
76      Air India Limited
Name: Operating Airline, Length: 77, dtype: object

Aparecen 77 compañías diferentes

> ¿Cuántos pasajeros tienen de media los vuelos de cada compañía?

In [24]:
df.groupby("Operating Airline")["Adjusted Passenger Count"].mean().compute()

Operating Airline
ATA Airlines          9661.659091
Aer Lingus            4407.183673
Aeromexico            5463.822222
Air Berlin            2320.750000
Air Canada           18251.560109
                         ...     
Virgin Atlantic       9847.104651
WestJet Airlines      5338.155340
World Airways          261.666667
XL Airways France     2240.129032
Xtra Airways            73.000000
Name: Adjusted Passenger Count, Length: 77, dtype: float64

Es creible porque según los datos de esa parte del dataset, el conteo de pasajeros por vuelo es muy alto

> Eliminaremos los registros duplicados por el campo “GEO Región”, manteniendo
únicamente aquel con mayor número de pasajeros.

In [25]:
print(f"({df.shape[0].compute()},{df.shape[1]})")

(15007,102)


In [26]:
# Eliminaremos los registros duplicados por el campo “GEO Región”, manteniendo únicamente aquel con mayor número de pasajeros.
df = df.drop_duplicates(subset=["GEO Region"], keep="last")

In [27]:
print(f"({df.shape[0].compute()},{df.shape[1]})")

(9,102)


Nos hemos cargado los datos totalmente